# Mapping NY State: Population Density with Kepler GL

In this notebook, we'll tie together the US Census population and geographic data we cleaned in previous project notebooks to visually map population density across NY State, highlighting the boundaries of New York City. For mapping, we'll make use of the Kepler GL plugin for Jupyter Environment. While these final steps are certainly more colorful and visually compelling than those in the previous notebooks, they actually represent the *easiest part of the process!* It was the somewhat tedious, strangely satisfying tasks of gathering and cleaning data, detailed in previous notebooks, that took the lion's share of time and effort. 

#### *Up close: data cleaning and mapping*

The notebooks below provide a detailed look at the other stages of cleaning and mapping the data for this project. The final steps of creating a SQL database of indoor farms in New York City and then mapping them will follow shortly.

* [Introduction: The State of Indoor Farming in the US](00_IndoorAgriculture_start.ipynb) 
* [Data Cleanup: US Census population data](01_DataCleanup_Population.ipynb) 
* [Data Cleanup: GEOID tables](02_DataCleanup_GEOIDs.ipynb)
* [Data Cleanup: Geographic data](03_DataCleanup_GIS.ipynb)
<!-- * [Mapping NY State: Population Density](http://localhost:8888/lab/tree/notebooks/04_Mapping_Population.ipynb) -->
* SQL Database Creation: Indoor Farms - *coming soon...*
* Mapping NY State: Indoor Farms - *coming soon...*

#### *Some helper code for managing file paths*
We'll use the code below throughout this project to make it easier to refer to the folders where our various data files are stored.

In [1]:
# os and pathlib modules used to make it easier to refer to project folders 

import os, pathlib
base_dir = pathlib.Path(os.getcwd()).parent
data_archive_dir = os.path.join(base_dir, "data_archive")
clean_data_dir = os.path.join(data_archive_dir, "clean")
data_dir = os.path.join(base_dir, "data")
shapes_dir = os.path.join(data_dir,"shapes")
json_dir = os.path.join(data_dir,"geojson")
util_dir = os.path.join(data_dir,"util")

#### *Imports*

In [2]:
import pandas as pd
import geopandas as gpd

In [3]:
# import NY State Census Tracts GEO DataFrame from CSV file
clean_geodf_tract_ny_file_csv = os.path.join(clean_data_dir,'tl_2020_36_tract20.csv')
geodf_tract_ny = pd.read_csv(clean_geodf_tract_ny_file_csv, index_col=0)
geodf_tract_ny.head(3)

,County FIPS,Census Tract Name,GEOID Census Tract,Population,Land Area,Population Density,geometry
State FIPS,,,,,,,
36,47,"Census Tract 7, Kings County, New York",36047000700,4415,176774,64686,"POLYGON ((-74.001542 40.692786, -74.001317 40...."
36,47,"Census Tract 9, Kings County, New York",36047000900,5167,163469,81865,"POLYGON ((-73.994046 40.690897, -73.993736 40...."
36,47,"Census Tract 11, Kings County, New York",36047001100,1578,168507,24254,"POLYGON ((-73.99072799999999 40.693048, -73.99..."


In [4]:
# we want to decrease file size to speed up mapping, this is why we saved our
# GEO DataFrames as Parquet files, but the geometry data gets mangled when saving to 
# Parquet, so we'll load from CSV files, until we work out the Parquet issue

# import NY State Census Tracts GEO DataFrame from Parquet file
# import pyarrow.parquet as pq
# clean_geodf_tract_ny_file = os.path.join(clean_data_dir, 'tl_2020_36_tract20.parquet')
# geodf_tract_ny = pq.read_table(clean_geodf_tract_ny_file, memory_map=True).to_pandas()
# geodf_tract_ny.head(3)

In [5]:
# load GEO DataFrame of the "Place" New York City from CSV file
clean_geodf_place_nyc_file_csv = os.path.join(clean_data_dir,'geodf_place_nyc.csv')
geodf_place_nyc = pd.read_csv(clean_geodf_place_nyc_file_csv, index_col=0)
geodf_place_nyc

,State,State FIPS,Place,Place FIPS,County,Land Area,geometry
State Name,,,,,,,
New York,NY,36,New York City,51000,"Bronx County, Kings County, New York County, Q...",778165283,"MULTIPOLYGON (((-74.040747 40.700167, -74.0407..."


In [6]:
from keplergl import KeplerGl

ny_map = KeplerGl(height=600, show_docs=False)
ny_map.add_data(data=geodf_place_nyc, name='New York City') 
ny_map.add_data(data=geodf_tract_ny, name= 'Population Density')

In [7]:
ny_map.config = {
  'version': 'v1',
'config': {'visState': {'filters': [],
 'layers': [{'id': 'hjl4aki',
   'type': 'geojson',
   'config': {'dataId': 'New York City',
    'label': 'New York City',
    'color': [173, 185, 51],
    'highlightColor': [252, 242, 26, 255],
    'columns': {'geojson': 'geometry'},
    'isVisible': True,
    'visConfig': {'opacity': 0,
     'strokeOpacity': 0.8,
     'thickness': 1.0,
     'strokeColor': [173, 185, 51],
     'colorRange': {'name': 'Global Warming',
      'type': 'sequential',
      'category': 'Uber',
      'colors': ['#5A1846',
       '#900C3F',
       '#C70039',
       '#E3611C',
       '#F1920E',
       '#FFC300']},
     'strokeColorRange': {'name': 'Global Warming',
      'type': 'sequential',
      'category': 'Uber',
      'colors': ['#5A1846',
       '#900C3F',
       '#C70039',
       '#E3611C',
       '#F1920E',
       '#FFC300']},
     'radius': 10,
     'sizeRange': [0, 10],
     'radiusRange': [0, 50],
     'heightRange': [0, 500],
     'elevationScale': 5,
     'enableElevationZoomFactor': True,
     'stroked': True,
     'filled': True,
     'enable3d': False,
     'wireframe': False},
    'hidden': False,
    'textLabel': [{'field': None,
      'color': [255, 255, 255],
      'size': 18,
      'offset': [0, 0],
      'anchor': 'start',
      'alignment': 'center'}]},
   'visualChannels': {'colorField': None,
    'colorScale': 'quantile',
    'strokeColorField': None,
    'strokeColorScale': 'quantile',
    'sizeField': None,
    'sizeScale': 'linear',
    'heightField': None,
    'heightScale': 'linear',
    'radiusField': None,
    'radiusScale': 'linear'}},
  {'id': 'ge5p5d',
   'type': 'geojson',
   'config': {'dataId': 'Population Density',
    'label': 'Population Density',
    'color': [255, 203, 153],
    'highlightColor': [252, 242, 26, 255],
    'columns': {'geojson': 'geometry'},
    'isVisible': True,
    'visConfig': {'opacity': 0.8,
     'strokeOpacity': 0.8,
     'thickness': 0,
     'strokeColor': [248, 149, 112],
     'colorRange': {'name': 'Global Warming',
      'type': 'sequential',
      'category': 'Uber',
      'colors': ['#5A1846',
       '#900C3F',
       '#C70039',
       '#E3611C',
       '#F1920E',
       '#FFC300']},
     'strokeColorRange': {'name': 'Global Warming',
      'type': 'sequential',
      'category': 'Uber',
      'colors': ['#5A1846',
       '#900C3F',
       '#C70039',
       '#E3611C',
       '#F1920E',
       '#FFC300']},
     'radius': 10,
     'sizeRange': [0, 10],
     'radiusRange': [0, 50],
     'heightRange': [0, 500],
     'elevationScale': 5,
     'enableElevationZoomFactor': True,
     'stroked': True,
     'filled': True,
     'enable3d': False,
     'wireframe': False},
    'hidden': False,
    'textLabel': [{'field': None,
      'color': [255, 255, 255],
      'size': 18,
      'offset': [0, 0],
      'anchor': 'start',
      'alignment': 'center'}]},
   'visualChannels': {'colorField': {'name': 'Population Density',
     'type': 'integer'},
    'colorScale': 'quantile',
    'strokeColorField': None,
    'strokeColorScale': 'quantile',
    'sizeField': None,
    'sizeScale': 'linear',
    'heightField': None,
    'heightScale': 'linear',
    'radiusField': None,
    'radiusScale': 'linear'}}],
 'interactionConfig': {'tooltip': {'fieldsToShow': {'New York City': [{'name': 'State',
      'format': None},
     {'name': 'State FIPS', 'format': None},
     {'name': 'Place', 'format': None},
     {'name': 'Place FIPS', 'format': None},
     {'name': 'County', 'format': None}],
    'Population Density': [{'name': 'County FIPS', 'format': None},
     {'name': 'Census Tract Name', 'format': None},
     {'name': 'GEOID Census Tract', 'format': None},
     {'name': 'Population', 'format': None},
     {'name': 'Land Area', 'format': None}]},
   'compareMode': False,
   'compareType': 'absolute',
   'enabled': True},
  'brush': {'size': 0.5, 'enabled': False},
  'geocoder': {'enabled': False},
  'coordinate': {'enabled': False}},
 'layerBlending': 'normal',
 'splitMaps': [],
 'animationConfig': {'currentTime': None, 'speed': 1}},
'mapState': {'bearing': 0,
 'dragRotate': False,
 'latitude': 40.70679298375131,
 'longitude': -74.0709215373596,
 'pitch': 0,
 'zoom': 9,
 'isSplit': False},
'mapStyle': {'styleType': 'dark',
 'topLayerGroups': {},
 'visibleLayerGroups': {'label': True,
  'road': True,
  'border': False,
  'building': True,
  'water': True,
  'land': True,
  '3d building': False},
 'threeDBuildingColor': [9.665468314072013,
  17.18305478057247,
  31.1442867897876],
 'mapStyles': {}}}}

In [8]:
ny_map

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'hjl4aki', 'type': …